In [8]:
import numpy as np
import jax
import jax.numpy as jnp
import haiku as hk

In [28]:
key = jax.random.PRNGKey(42)
mask = jax.random.bernoulli(key, 0.5, shape=[10, 10])

In [29]:
jnp.ones([10, 10], dtype=jnp.float32) * mask

DeviceArray([[0., 0., 0., 1., 1., 1., 1., 0., 1., 1.],
             [0., 1., 0., 0., 0., 0., 0., 1., 1., 1.],
             [1., 1., 0., 0., 0., 1., 0., 1., 1., 0.],
             [1., 1., 1., 1., 1., 0., 0., 1., 1., 1.],
             [0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
             [1., 0., 0., 0., 1., 1., 1., 0., 0., 1.],
             [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
             [1., 0., 1., 0., 1., 1., 1., 0., 0., 1.],
             [0., 0., 1., 0., 0., 0., 0., 1., 1., 1.],
             [0., 1., 1., 1., 1., 1., 1., 1., 0., 1.]], dtype=float32)

In [64]:
class Test(hk.Module):
    def __init__(self):
        # apply 2 sets of layer norm to test Haiku behaviour.
        super().__init__()
    
    def __call__(self, x):
        x = hk.LayerNorm(axis=-1, create_scale=True, create_offset=True)(x)
        x = hk.LayerNorm(axis=-1, create_scale=True, create_offset=True)(x)
        return x
    
class Test2(hk.Module):
    def __init__(self):
        # apply 2 sets of layer norm to test Haiku behaviour.
        super().__init__()
    
    def __call__(self, x):
        ln_reuse = hk.LayerNorm(axis=-1, create_scale=True, create_offset=True)
        x = ln_reuse(x)
        x = ln_reuse(x)
        return x
    
def net(x):
    return Test2()(x)

In [65]:
# get random key
rng_key = jax.random.PRNGKey(42)

# transform fn. with state
f = hk.transform_with_state(net)
f = hk.without_apply_rng(f)

dummy = np.random.randn(10, 100).astype(np.float32)
params, state = f.init(rng_key, dummy)

In [66]:
print(params)

FlatMap({
  'test2/layer_norm': FlatMap({
                        'scale': DeviceArray([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
                                              1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32),
                        'offset': DeviceArray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                                               0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
     